In [1]:
%%time
!python -m pip install -q --no-index --find-links=/kaggle/input/eedi-dependencies/dependencies-tv/ -r /kaggle/input/eedi-dependencies/requirements-tv.txt
!python -m pip install -q --no-index --find-links=/kaggle/input/eedi-dependencies/dependencies-vllm/ -r /kaggle/input/eedi-dependencies/requirements-vllm.txt
!pip install --no-deps --no-index /kaggle/input/logits-processor-zoo/logits_processor_zoo-0.1.0-py3-none-any.whl

Processing /kaggle/input/logits-processor-zoo/logits_processor_zoo-0.1.0-py3-none-any.whl
CPU times: user 2.08 s, sys: 532 ms, total: 2.62 s
Wall time: 3min 25s


In [2]:
%%time

!cp -r /kaggle/input/eedi-dependencies/AutoAWQ ./AutoAWQ
!pip install ./AutoAWQ

Processing ./AutoAWQ
  Preparing metadata (setup.py) ... done
  Created wheel for autoawq: filename=autoawq-0.2.7.post2-py3-none-any.whl size=106958 sha256=33abab1f20c6977409eb47935bcaf5c5d71c84ba345f4f2f93533fdc3927f7e1
  Stored in directory: /tmp/pip-ephem-wheel-cache-ishw80iq/wheels/e1/d4/ec/8ae90063250f3f09fc58d9ea8b2026549874540925fa9ed089
Successfully built autoawq
CPU times: user 447 ms, sys: 136 ms, total: 582 ms
Wall time: 48.6 s


In [3]:
!pip list | grep -e vllm -e torch -e transformers

pytorch-ignite                           0.5.1
pytorch-lightning                        2.4.0
torch                                    2.4.0
torchaudio                               2.4.0
torchinfo                                1.8.0
torchmetrics                             1.6.0
torchvision                              0.19.0
transformers                             4.46.2
vllm                                     0.6.3


## Metric

In [4]:
%%writefile eedi_metrics.py

# Credit: https://www.kaggle.com/code/abdullahmeda/eedi-map-k-metric

import numpy as np
def apk(actual, predicted, k=25):
    """
    Computes the average precision at k.
    
    This function computes the average prescision at k between two lists of
    items.
    
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
        
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    
    if not actual:
        return 0.0

    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        # first condition checks whether it is valid prediction
        # second condition checks if prediction is not repeated
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    return score / min(len(actual), k)

def mapk(actual, predicted, k=25):
    """
    Computes the mean average precision at k.
    
    This function computes the mean average prescision at k between two lists
    of lists of items.
    
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
        
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

Writing eedi_metrics.py


# first retrieval

In [5]:
import os, math, numpy as np
import sys
import os
from transformers import AutoTokenizer
import pandas as pd
from tqdm import tqdm
import re, gc
import torch
pd.set_option('display.max_rows', 300)

In [6]:
IS_SUBMISSION = bool(os.getenv("KAGGLE_IS_COMPETITION_RERUN"))


print('IS_SUBMISSION:', IS_SUBMISSION)

model_path = "/kaggle/input/qwen2.5/transformers/32b-instruct-awq/1"
df_train = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/train.csv").fillna(-1).sample(10, random_state=42).reset_index(drop=True)
df_test = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/test.csv")
df_misconception_mapping = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv")

IS_SUBMISSION: False


In [7]:
import pandas as pd
# from sentence_transformers import SentenceTransformer, util
if not IS_SUBMISSION:
    df_ret = df_train.copy()
else:
    df_ret = df_test.copy()

df_ret

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText,MisconceptionAId,MisconceptionBId,MisconceptionCId,MisconceptionDId
0,1700,321,Divide two decimals with the same number of de...,224,Multiplying and Dividing with Decimals,D,\( 0.9 \div 0.3= \),\( 0.3 \),\( 9.3 \),\( 0.333 \ldots \),\( 3 \),153.0,-1.0,2359.0,-1.0
1,1488,469,Understand the notation for powers,245,"Squares, Cubes, etc",C,To calculate \( 53^{2} \) you need to do...,\( 53+2 \),\( 53 \times 2 \),\( 53 \times 53 \),\( 532 \times 1 \),2445.0,2316.0,-1.0,-1.0
2,921,219,Round numbers to three or more decimal places,214,Rounding to Decimal Places,A,What is \( 20.15349 \) rounded to \( 3 \) deci...,\( 20.153 \),\( 20.15 \),\( 20.154 \),\( 20.253 \),-1.0,2392.0,1988.0,2330.0
3,275,306,Subtract decimals where the numbers involved h...,223,Adding and Subtracting with Decimals,A,\( 50.09-0.1= \),\( 49.99 \),\( 50.99 \),\( 50.08 \),\( 38.98 \),-1.0,699.0,2346.0,-1.0
4,416,703,Express pictorial representations of objects a...,334,Writing Ratios,C,Tom says for every one circle there are two sq...,Only\nTom,Only Katie,Both Tom and Katie,Neither is correct,-1.0,1846.0,-1.0,1150.0
5,1654,1556,"Given a positive x value, find the correspondi...",57,Cubics and Reciprocals,A,This is a part of the table of values for the ...,\( 1 \),\( 3 \),\( \frac{1}{3} \),\( 0 \),-1.0,2349.0,-1.0,832.0
6,247,71,Solve problems that combine addition and subtr...,206,Written Subtraction,B,"At the start of the day, there was \( £ 1,456 ...","\( £ 13,180 \)","\( £ 4,426 \)","\( £ 2,970 \)","\( £ 10,268 \)",2009.0,-1.0,791.0,1787.0
7,383,1525,Substitute positive integer values into expres...,67,Substitution into Formula,B,When \( h=15 \)\n\n\(\n\frac{h}{3}=\n\)\n,\( 12 \),\( 5 \),\( -9 \),\( -5 \),2093.0,-1.0,-1.0,1648.0
8,322,311,Multiply a decimal by an integer,224,Multiplying and Dividing with Decimals,B,Work out:\n\[\n0.3 \times 12\n\],\( 36 \),\( 3.6 \),\( 0.36 \),\( 0.036 \),189.0,-1.0,1035.0,2187.0
9,203,1399,Calculate the distance between two coordinates...,410,Distance Between Two Co-ordinates,B,Mark is working out the distance between these...,Isosceles,Right-angled,Equilateral,Scalene,1658.0,-1.0,1658.0,1658.0


In [8]:
TEMPLATE_INPUT_V3 = '{QUESTION}\nCorrect answer: {CORRECT_ANSWER}\nStudent wrong answer: {STUDENT_WRONG_ANSWER}'
def format_input_v3(row, wrong_choice):

    assert wrong_choice in "ABCD"
    # Extract values from the row
    question_text = row.get("QuestionText", "No question text provided")
    subject_name = row.get("SubjectName", "Unknown subject")
    construct_name = row.get("ConstructName", "Unknown construct")
    # Extract the correct and wrong answer text based on the choice
    correct_answer = row.get("CorrectAnswer", "Unknown")
    assert wrong_choice != correct_answer
    correct_answer_text = row.get(f"Answer{correct_answer}Text", "No correct answer text available")
    wrong_answer_text = row.get(f"Answer{wrong_choice}Text", "No wrong answer text available")

    # Construct the question format
    formatted_question = f"""Question: {question_text}
    
SubjectName: {subject_name}
ConstructName: {construct_name}"""

    # Return the extracted data
    ret = {
        "QUESTION": formatted_question,
        "CORRECT_ANSWER": correct_answer_text,
        "STUDENT_WRONG_ANSWER": wrong_answer_text,
        "MISCONCEPTION_ID": row.get('Misconception{wrong_choice}Id'),
    }
    ret["PROMPT"] = TEMPLATE_INPUT_V3.format(**ret)

    return ret


items = []
target_ids = []
for _, row in df_ret.iterrows():
    for choice in ['A', 'B', 'C', 'D']:
        if choice == row["CorrectAnswer"]:
            continue
        if not IS_SUBMISSION and row[f'Misconception{choice}Id'] == -1:
            continue
            
        correct_col = f"Answer{row['CorrectAnswer']}Text"
        item = {'QuestionId_Answer': '{}_{}'.format(row['QuestionId'], choice)}
        item['Prompt'] = format_input_v3(row, choice)['PROMPT']
        items.append(item)
        target_ids.append(int(row.get(f'Misconception{choice}Id', -1)))
        
df_input = pd.DataFrame(items)

In [9]:
def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'<instruct>{task_description}\n<query>{query}'

def get_detailed_example(task_description: str, query: str, response: str) -> str:
    return f'<instruct>{task_description}\n<query>{query}\n<response>{response}'

def get_new_queries(queries, query_max_len, examples_prefix, tokenizer):
    inputs = tokenizer(
        queries,
        max_length=query_max_len - len(tokenizer('<s>', add_special_tokens=False)['input_ids']) - len(
            tokenizer('\n<response></s>', add_special_tokens=False)['input_ids']),
        return_token_type_ids=False,
        truncation=True,
        return_tensors=None,
        add_special_tokens=False
    )
    prefix_ids = tokenizer(examples_prefix, add_special_tokens=False)['input_ids']
    suffix_ids = tokenizer('\n<response>', add_special_tokens=False)['input_ids']
    new_max_length = (len(prefix_ids) + len(suffix_ids) + query_max_len + 8) // 8 * 8 + 8
    new_queries = tokenizer.batch_decode(inputs['input_ids'])
    for i in range(len(new_queries)):
        new_queries[i] = examples_prefix + new_queries[i] + '\n<response>'
    return new_max_length, new_queries
task =  "Given a math multiple-choice problem with a student's wrong answer, retrieve the math misconceptions"
queries = [
    get_detailed_instruct(task, q) for q in df_input['Prompt']
]
documents = df_misconception_mapping['MisconceptionName'].tolist()
query_max_len, doc_max_len = 320, 48
LORA_PATH = '/kaggle/input/2211-lora-14b/transformers/default/1'
tokenizer = AutoTokenizer.from_pretrained(LORA_PATH)
examples_prefix = ''
new_query_max_len, new_queries = get_new_queries(queries, query_max_len, examples_prefix, tokenizer)


import json
with open('data.json', 'w') as f:
    data = {'texts': new_queries+ documents}
    f.write(json.dumps(data))

In [10]:
new_query_max_len

336

In [11]:
%%writefile run_embed.py
import argparse
import os
import json
import torch
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
import peft

MAX_LENGTH = 320


def last_token_pool(last_hidden_states: Tensor, attention_mask: Tensor) -> Tensor:
    left_padding = attention_mask[:, -1].sum() == attention_mask.shape[0]
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[
            torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths
        ]


def get_embeddings_in_batches(model, tokenizer, texts, max_length, batch_size=32):
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Embedding"):
        batch_texts = texts[i : i + batch_size]
        batch_dict = tokenizer(
            batch_texts,
            max_length=max_length,
            padding=True,
            truncation=True,
            return_tensors="pt",
        ).to("cuda")
        with torch.no_grad(), torch.amp.autocast("cuda"):
            outputs = model(**batch_dict)
            batch_embeddings = last_token_pool(
                outputs.last_hidden_state, batch_dict["attention_mask"]
            )
            batch_embeddings = F.normalize(batch_embeddings, p=2, dim=1).cpu()
        embeddings.append(batch_embeddings)
    return torch.cat(embeddings, dim=0)


def load_model_and_tokenizer(base_model_path, lora_path, load_in_4bit=True):
    model = AutoModel.from_pretrained(
        base_model_path,
        device_map=0,
        torch_dtype=torch.float16,
        load_in_4bit=load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained(
        lora_path if lora_path else base_model_path
    )
    model.resize_token_embeddings(len(tokenizer))
    if lora_path:
        model = peft.PeftModel.from_pretrained(model, lora_path)
    return model, tokenizer


def main(args):
    output_file = args.input_text.replace(
        ".json", ".pt.fold.{}.{}.embed".format(*args.fold)
    )
    if os.path.exists(output_file):
        print(f"Output file {output_file} already exists. Skipping...")
        return
    model, tokenizer = load_model_and_tokenizer(
        args.base_model, args.lora_path, load_in_4bit=args.load_in_4bit
    )
    texts = json.load(open(args.input_text))["texts"][args.fold[0] :: args.fold[1]]
    embeddings = get_embeddings_in_batches(
        model,
        tokenizer,
        texts,
        max_length=MAX_LENGTH,
        batch_size=4,
    )
    text2embeds = {text: emb for text, emb in zip(texts, embeddings)}
    torch.save(text2embeds, output_file)


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--base_model",
        type=str,
        default="Qwen/Qwen2.5-7B",
        help="Path to the base model",
    )
    parser.add_argument(
        "--lora_path",
        type=str,
        default=None,
        help="Path to the LoRA model",
    )
    parser.add_argument(
        "--input_text",
        type=str,
        default=".cache/data.json",
    )
    parser.add_argument(
        "--load_in_4bit",
        action="store_true",
        help="Load model in 4-bit mode",
    )
    parser.add_argument("--fold", nargs=2, type=int, default=[0, 1])
    args = parser.parse_args()
    if not os.path.exists(args.lora_path):
        args.lora_path = None
    main(args)


Writing run_embed.py


In [12]:
!sleep 1 & sleep 3

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:
lora_path = '/kaggle/input/2211-lora-14b/transformers/default/1'
cmd = f"(CUDA_VISIBLE_DEVICES=0 python run_embed.py --base_model /kaggle/input/qw14b-awq/transformers/default/1 --lora_path {lora_path} --input_text data.json --fold 0 2) & (CUDA_VISIBLE_DEVICES=1 python run_embed.py --base_model /kaggle/input/qw14b-awq/transformers/default/1 --lora_path {lora_path} --input_text data.json --fold 1 2)"
import os
os.system(cmd)

Embedding: 100%|██████████| 327/327 [03:36<00:00,  1.51it/s]


0

In [14]:
! sleep 10

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [15]:
from glob import glob
import time
text_to_embed = {}
files = glob('*.pt*')
while len(files) != 2:
    time.sleep(1)
    files = glob('*.pt*')


time.sleep(3)    
for path in files:
    print(path)
    text_to_embed.update(torch.load(path))

data.pt.fold.1.2.embed
data.pt.fold.0.2.embed


/tmp/ipykernel_23/3626513216.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  text_to_embed.update(torch.load(path))


In [16]:
query_embeddings = torch.stack([text_to_embed[t] for t in new_queries])
doc_embeddings = torch.stack([text_to_embed[t] for t in documents])
query_embeddings.shape, doc_embeddings.shape

(torch.Size([24, 5120]), torch.Size([2587, 5120]))

In [17]:
scores = query_embeddings @ doc_embeddings.T  # Shape: (M, N)
sorted_indices = torch.argsort(scores,1, descending=True)[:,:25].tolist()
df_input["MisconceptionId"] = [" ".join([str(x) for x in row]) for row in sorted_indices]
df_input[["QuestionId_Answer", "MisconceptionId"]].to_csv("submission.csv", index=False)

pd.read_csv('submission.csv')

,QuestionId_Answer,MisconceptionId
0,1700_A,2359 151 2525 1155 1074 153 565 195 2461 1922 ...
1,1700_C,2359 151 419 1155 2525 2542 1629 339 1259 58 6...
2,1488_A,948 2445 952 2175 2352 2316 1072 1337 2010 923...
3,1488_B,1072 2175 2316 1792 2445 948 1337 2010 1346 14...
4,921_B,1529 2392 1248 1379 1355 1272 739 663 1742 233...
5,921_C,1988 322 2392 2330 1379 2412 1248 794 1449 152...
6,921_D,2330 1675 2392 1248 1742 1988 2412 1529 322 13...
7,275_B,2346 496 699 2203 611 1730 1795 980 1898 467 1...
8,275_C,2346 496 2203 90 539 1730 699 980 2044 1317 18...
9,416_B,1846 843 1556 2373 1864 1224 473 1232 105 1895...


In [18]:
indices = np.array(sorted_indices, dtype=int)
np.save("indices.npy", indices)

## Prepare dataframe

In [19]:
import pandas as pd
import os

IS_SUBMISSION = bool(os.getenv("KAGGLE_IS_COMPETITION_RERUN"))
if IS_SUBMISSION:
    full_df = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/test.csv")
else:
    full_df = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/train.csv").fillna(-1).sample(10, random_state=42).reset_index(drop=True)



rows = []
for idx, row in full_df.iterrows():
    for option in ["A", "B", "C", "D"]:
        if option == row.CorrectAnswer:
            continue

        if not IS_SUBMISSION and row[f"Misconception{option}Id"]==-1:
            continue
            
        correct_answer = row[f"Answer{row.CorrectAnswer}Text"]

        query_text =f"### SubjectName: {row['SubjectName']}\n### ConstructName: {row['ConstructName']}\n### Question: {row['QuestionText']}\n### Correct Answer: {correct_answer}\n### Misconcepte Incorrect answer: {option}.{row[f'Answer{option}Text']}"
        rows.append({"query_text": query_text, 
                     "QuestionId_Answer": f"{row.QuestionId}_{option}",
                     "ConstructName": row.ConstructName,
                     "SubjectName": row.SubjectName,
                     "QuestionText": row.QuestionText,
                     "correct_answer": correct_answer,
                     "incorrect_answer": row[f"Answer{option}Text"]
                     })

df = pd.DataFrame(rows)
df

,query_text,QuestionId_Answer,ConstructName,SubjectName,QuestionText,correct_answer,incorrect_answer
0,### SubjectName: Multiplying and Dividing with...,1700_A,Divide two decimals with the same number of de...,Multiplying and Dividing with Decimals,\( 0.9 \div 0.3= \),\( 3 \),\( 0.3 \)
1,### SubjectName: Multiplying and Dividing with...,1700_C,Divide two decimals with the same number of de...,Multiplying and Dividing with Decimals,\( 0.9 \div 0.3= \),\( 3 \),\( 0.333 \ldots \)
2,"### SubjectName: Squares, Cubes, etc\n### Cons...",1488_A,Understand the notation for powers,"Squares, Cubes, etc",To calculate \( 53^{2} \) you need to do...,\( 53 \times 53 \),\( 53+2 \)
3,"### SubjectName: Squares, Cubes, etc\n### Cons...",1488_B,Understand the notation for powers,"Squares, Cubes, etc",To calculate \( 53^{2} \) you need to do...,\( 53 \times 53 \),\( 53 \times 2 \)
4,### SubjectName: Rounding to Decimal Places\n#...,921_B,Round numbers to three or more decimal places,Rounding to Decimal Places,What is \( 20.15349 \) rounded to \( 3 \) deci...,\( 20.153 \),\( 20.15 \)
5,### SubjectName: Rounding to Decimal Places\n#...,921_C,Round numbers to three or more decimal places,Rounding to Decimal Places,What is \( 20.15349 \) rounded to \( 3 \) deci...,\( 20.153 \),\( 20.154 \)
6,### SubjectName: Rounding to Decimal Places\n#...,921_D,Round numbers to three or more decimal places,Rounding to Decimal Places,What is \( 20.15349 \) rounded to \( 3 \) deci...,\( 20.153 \),\( 20.253 \)
7,### SubjectName: Adding and Subtracting with D...,275_B,Subtract decimals where the numbers involved h...,Adding and Subtracting with Decimals,\( 50.09-0.1= \),\( 49.99 \),\( 50.99 \)
8,### SubjectName: Adding and Subtracting with D...,275_C,Subtract decimals where the numbers involved h...,Adding and Subtracting with Decimals,\( 50.09-0.1= \),\( 49.99 \),\( 50.08 \)
9,### SubjectName: Writing Ratios\n### Construct...,416_B,Express pictorial representations of objects a...,Writing Ratios,Tom says for every one circle there are two sq...,Both Tom and Katie,Only Katie


In [20]:
if not IS_SUBMISSION:
    df_label = {}
    for idx, row in full_df.iterrows():
        for option in ["A", "B", "C", "D"]:
            if (row.CorrectAnswer!=option) & (row[f"Misconception{option}Id"]!=-1):
                df_label[f"{row.QuestionId}_{option}"] = [row[f"Misconception{option}Id"]]             
                
    df_label = pd.DataFrame([df_label]).T.reset_index()
    df_label.columns = ["QuestionId_Answer", "MisconceptionId"]
    df_label.to_parquet("label.parquet", index=False)
    print(df_label)

   QuestionId_Answer MisconceptionId
0             1700_A         [153.0]
1             1700_C        [2359.0]
2             1488_A        [2445.0]
3             1488_B        [2316.0]
4              921_B        [2392.0]
5              921_C        [1988.0]
6              921_D        [2330.0]
7              275_B         [699.0]
8              275_C        [2346.0]
9              416_B        [1846.0]
10             416_D        [1150.0]
11            1654_B        [2349.0]
12            1654_D         [832.0]
13             247_A        [2009.0]
14             247_C         [791.0]
15             247_D        [1787.0]
16             383_A        [2093.0]
17             383_D        [1648.0]
18             322_A         [189.0]
19             322_C        [1035.0]
20             322_D        [2187.0]
21             203_A        [1658.0]
22             203_C        [1658.0]
23             203_D        [1658.0]


In [21]:
df.to_parquet("df.parquet", index=False)

## LLM Reasoning

In [22]:
%%writefile run_vllm.py

import vllm
from vllm.lora.request import LoRARequest
import numpy as np
import pandas as pd
from transformers import PreTrainedTokenizer, AutoTokenizer
from typing import List
import torch
from logits_processor_zoo.vllm import MultipleChoiceLogitsProcessor
import re

model_path = "/kaggle/input/qwen32b/qwen-awq"
lora_path = '/kaggle/input/eedi-gather/lora/qwen2-32b/last_checkpoint/'

tokenizer = AutoTokenizer.from_pretrained(model_path)


def preprocess_text(x):
    x = re.sub("http\w+", '',x)   # Delete URL
    x = re.sub(r"\.+", ".", x)    # Replace consecutive commas and periods with one comma and period character
    x = re.sub(r"\,+", ",", x)
    x = re.sub(r"\\\(", " ", x)
    x = re.sub(r"\\\)", " ", x)
    x = re.sub(r"[ ]{1,}", " ", x)
    x = x.strip()                 # Remove empty characters at the beginning and end
    return x

PROMPT  = """Here is a question about {ConstructName}({SubjectName}).
Question: {Question}
Correct Answer: {CorrectAnswer}
Incorrect Answer: {IncorrectAnswer}

You are a Mathematics teacher. Your task is to reason and identify the misconception behind the Incorrect Answer with the Question.
Answer concisely what misconception it is to lead to getting the incorrect answer.
Pick the correct misconception number from the below:

{Retrival}
"""
# just directly give your answers.

def apply_template(row, tokenizer):
    messages = [
        {
            "role": "user", 
            "content": preprocess_text(
                PROMPT.format(
                    ConstructName=row["ConstructName"],
                    SubjectName=row["SubjectName"],
                    Question=row["QuestionText"],
                    IncorrectAnswer=row["incorrect_answer"],
                    CorrectAnswer=row["correct_answer"],
                    Retrival=row["retrieval"]
                )
            )
        }
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return text


misconception_df = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv")

df = pd.read_parquet("df.parquet")
indices = np.load("indices.npy")

model_path = "/kaggle/input/qwen32b/qwen-awq"

llm = vllm.LLM(
    model_path,
    quantization="awq",
    tensor_parallel_size=2,
    gpu_memory_utilization=0.90, 
    trust_remote_code=True,
    dtype="half", 
    enforce_eager=True,
    max_model_len=5120,
    disable_log_stats=True,
    enable_prefix_caching=True,
    enable_lora=True,
    max_lora_rank=64,
    max_loras=1
)
tokenizer = llm.get_tokenizer()


def get_candidates(c_indices):
    candidates = []

    mis_names = misconception_df["MisconceptionName"].values
    for ix in c_indices:
        c_names = []
        for i, name in enumerate(mis_names[ix]):
            c_names.append(f"{i+1}. {name}")

        candidates.append("\n".join(c_names))
        
    return candidates


# cache
select_rounds = 3
single_round_len = 24 // select_rounds
numbers = ["1", "2", "3", "4", "5", "6", "7", "8", "9"]
print(f'select_rounds: {select_rounds}, single_round_len: {single_round_len+1}')
print(numbers[:single_round_len+1])

sample_r = get_candidates(indices[0:1,0:1])
print(sample_r)
messages = [
    {
        "role": "user", 
        "content": preprocess_text(
            PROMPT.format(
                ConstructName=df.loc[0, "ConstructName"],
                SubjectName=df.loc[0, "SubjectName"],
                Question=df.loc[0, "QuestionText"],
                IncorrectAnswer=df.loc[0, "incorrect_answer"],
                CorrectAnswer=df.loc[0, "correct_answer"],
                Retrival=sample_r
            )
        )
    }
]
text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(text[0])

llm.generate(
    text[0],
    vllm.SamplingParams(
        n=1,  # Number of output sequences to return for each prompt.
        top_k=1,  # Float that controls the cumulative probability of the top tokens to consider.
        temperature=0,  # randomness of the sampling
        seed=777, # Seed for reprodicibility
        skip_special_tokens=False,  # Whether to skip special tokens in the output.
        max_tokens=1,  # Maximum number of tokens to generate per output sequence.
        logits_processors=[MultipleChoiceLogitsProcessor(tokenizer, choices=numbers[:single_round_len+1])]
    ),
    use_tqdm=True,
    lora_request=LoRARequest("myadapter", 1, lora_path)
)


# loop
survivors = indices[:, -1:]

for i in range(select_rounds):
    c_indices = np.concatenate([indices[:, -single_round_len*(i+1)-1:-single_round_len*i-1], survivors], axis=1)
    
    df["retrieval"] = get_candidates(c_indices)
    df["text"] = df.apply(lambda row: apply_template(row, tokenizer), axis=1)
    
    print("Example:")
    print(df["text"].values[0])
    print()
    
    responses = llm.generate(
        df["text"].values,
        vllm.SamplingParams(
            n=1,  # Number of output sequences to return for each prompt.
            top_k=1,  # Float that controls the cumulative probability of the top tokens to consider.
            temperature=0,  # randomness of the sampling
            seed=777, # Seed for reprodicibility
            skip_special_tokens=False,  # Whether to skip special tokens in the output.
            max_tokens=1,  # Maximum number of tokens to generate per output sequence.
            logits_processors=[MultipleChoiceLogitsProcessor(tokenizer, choices=numbers[:single_round_len+1])]
        ),
        use_tqdm=True,
        lora_request=LoRARequest("myadapter", 1, lora_path)
    )
    
    responses = [x.outputs[0].text for x in responses]
    df["response"] = responses
    
    
    llm_choices = df["response"].astype(int).values - 1
    
    survivors = np.array([cix[best] for best, cix in zip(llm_choices, c_indices)]).reshape(-1, 1)



results = []

for i in range(indices.shape[0]):
    ix = indices[i]
    llm_choice = survivors[i, 0]
    
    results.append(" ".join([str(llm_choice)] + [str(x) for x in ix if x != llm_choice]))


df["MisconceptionId"] = results
df.to_csv("submission.csv", columns=["QuestionId_Answer", "MisconceptionId"], index=False)

Writing run_vllm.py


In [23]:
!python run_vllm.py

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


WARNING 12-12 15:05:00 cuda.py:22] You are using a deprecated `pynvml` package. Please install `nvidia-ml-py` instead, and make sure to uninstall `pynvml`. When both of them are installed, `pynvml` will take precedence and cause errors. See https://pypi.org/project/pynvml for more information.
/opt/conda/lib/python3.10/site-packages/vllm/connections.py:8: RuntimeWarning: Failed to read commit hash:
No module named 'vllm._version'
  from vllm.version import __version__ as VLLM_VERSION
WARNING 12-12 15:05:15 config.py:306] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 12-12 15:05:15 config.py:887] Defaulting to use mp for distributed inference
WARNING 12-12 15:05:15 config.py:380] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 12-12 15:05:15 llm_engine.py:237] Initializing an LLM engine (vdev) with config: model='/kaggle/input/qwen32b/qwen-awq

In [24]:
pd.read_csv("submission.csv")

,QuestionId_Answer,MisconceptionId
0,1700_A,153 2359 151 2525 1155 1074 565 195 2461 1922 ...
1,1700_C,2359 151 419 1155 2525 2542 1629 339 1259 58 6...
2,1488_A,2445 948 952 2175 2352 2316 1072 1337 2010 923...
3,1488_B,2316 1072 2175 1792 2445 948 1337 2010 1346 14...
4,921_B,2392 1529 1248 1379 1355 1272 739 663 1742 233...
5,921_C,1988 322 2392 2330 1379 2412 1248 794 1449 152...
6,921_D,2330 1675 2392 1248 1742 1988 2412 1529 322 13...
7,275_B,699 2346 496 2203 611 1730 1795 980 1898 467 1...
8,275_C,2346 496 2203 90 539 1730 699 980 2044 1317 18...
9,416_B,1846 843 1556 2373 1864 1224 473 1232 105 1895...


## Sanity

In [25]:
if not IS_SUBMISSION:
    import pandas as pd
    from eedi_metrics import mapk
    predicted = pd.read_csv("submission.csv")["MisconceptionId"].apply(lambda x: [int(y) for y in x.split()])
    label = pd.read_parquet("label.parquet")["MisconceptionId"]
    print("Validation: ", mapk(label, predicted))

Validation:  0.8784722222222222
